# Problem 2: Determine the Phase Transformation Pressure

NOTICE TO BINDER USERS: YOUR NOTEBOOK PROGRESS WILL NOT BE SAVED IF YOU CLOSE THIS WINDOW OR LEAVE IT INACTIVE FOR TOO LONG.

PLEASE DOWNLOAD YOUR NOTEBOOKS AND FILES REGULARLY OR DOWNLOAD THIS REPO AND RUN OFFLINE ON YOUR MACHINE. See "running_offline.md" for more info

## Applying strains to structures with pymatgen

For this walkthrough, I'll start with the relaxed structures for our two phases from the Materials Project (found in the directory "mp_structures") but you should use the structures that were output by your VASP calculations in Problem 1.

In [6]:
from pymatgen import Structure
from pymatgen.io.vasp import Poscar
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

# Si_DC = Structure.from_file("YOUR OPTIMIZATION CALC DIRECTORY")
Si_DC = Structure.from_file("mp_structures/Si_DC_Primitive.POSCAR")
Si_BSn = Structure.from_file("mp_structures/Si_BSn_Primitive.POSCAR")

Now, we'll need to create some VASP input files for our E vs V calculations. The first step is to have some way of applying uniform strains to structures. Luckily, pymatgen can do this for us too with it's `Deformation` class. Let's create a `Deformation` that applies a uniform strain of 10% to a structure. 

In [8]:
from pymatgen.io.vasp.inputs import Kpoints, Poscar, Potcar, Incar
from pymatgen.analysis.elasticity.strain import Deformation

#Define our strain tensor. In this case, e_11 = e_22 = e_33 = 1.10.
strain_tensor = ((1.10, 0, 0), (0, 1.10, 0), (0, 0, 1.10))

deformation = Deformation(strain_tensor)

Let's apply this example deformation to our DC-Si structure and compare the before and after.

In [15]:
print(Si_DC)

Full Formula (Si2)
Reduced Formula: Si
abc   :   3.866975   3.866975   3.866975
angles:  60.000000  60.000000  60.000000
Sites (2)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Si    0     0     0
  1  Si    0.25  0.25  0.25


In [16]:
deformed_Si_DC = deformation.apply_to_structure(Si_DC)
print(deformed_Si_DC)

Full Formula (Si2)
Reduced Formula: Si
abc   :   4.253672   4.253672   4.253672
angles:  60.000000  60.000000  60.000000
Sites (2)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Si    0     0     0
  1  Si    0.25  0.25  0.25


As we can see, our structure's lattice parameters have been increased by 10%. You should now be able to use similar code to the functions you used in Problem 1 to write new VASP calculations using these deformed structures as inputs.

In [ ]:
# Your code here

## Fitting the Equation of State

In this walkthrough we'll be extending pymatgen's EOS class to fit a Murnaghan equation of state. To extend a class, we need to specify some of it's functionality by declaring class methods (functions that belong to a class.) EOSBase takes in a list of energies and volume and tries to fit a function to them. In our case, we want the function it fits to be the Murnaghan equation of state:

<img src="static/Murnaghan.png" alt="Murnaghan equation of state" width="300"/>


Below you will find the base class, EOSBase, which will be extending. Any function with "pass" in it instead of code will need to be defined by us in our `Murnaghan` class.

In [25]:
from abc import ABCMeta, abstractmethod
import numpy as np
from scipy.optimize import minimize
import scipy
from pymatgen.core.units import FloatWithUnit
from pymatgen.util.plotting import pretty_plot, add_fig_kwargs, get_ax_fig_plt

class EOSBase(metaclass=ABCMeta):
    """
    Abstract class that must be subcalssed by all equation of state
    implementations.
    """

    def __init__(self, volumes, energiesm, name=''):
        """
         Args:
             volumes (list/numpy.array): volumes in Ang^3
             energies (list/numpy.array): energy in eV
        """
        self.volumes = np.array(volumes)
        self.energies = np.array(energies)
        # minimum energy(e0), buk modulus(b0),
        # derivative of bulk modulus with respect to pressure(b1), minimum volume(v0)
        self._params = None
        # the eos function parameters. It is the same as _params except for
        # equation of states that uses polynomial fits(deltafactor and
        # numerical_eos)
        self.eos_params = None
        self.name = name

    def _initial_guess(self):
        """
        Quadratic fit to get an initial guess for the parameters.

        Returns:
            tuple: (e0, b0, b1, v0)
        """
        a, b, c = np.polyfit(self.volumes, self.energies, 2)
        self.eos_params = [a, b, c]

        v0 = -b/(2*a)
        e0 = a*(v0**2) + b*v0 + c
        b0 = 2 * a * v0
        b1 = 4  # b1 is usually a small number like 4

        vmin, vmax = min(self.volumes), max(self.volumes)

        if not vmin < v0 and v0 < vmax:
            raise EOSError('The minimum volume of a fitted parabola is '
                           'not in the input volumes\n.')

        return e0, b0, b1, v0

    def fit(self):
        """
        Do the fitting. Does least square fitting. If you want to use custom
        fitting, must override this. This must be implemented by all classes
        that derive from this abstract class.
        """
        pass

    @abstractmethod
    def _func(self, volume, params):
        """
        The equation of state function. This must be implemented by all classes
        that derive from this abstract class.

        Args:
            volume (float/numpy.array)
             params (list/tuple): values for the parameters other than the
                volume used by the eos.
        """
        pass

    def func(self, volume):
        """
        The equation of state function with the paramters other than volume set
        to the ones obtained from fitting.

        Args:
             volume (list/numpy.array)

        Returns:
            numpy.array
        """
        return self._func(np.array(volume), self.eos_params)


    def __call__(self, volume):
        return self.func(volume)

    @property
    def e0(self):
        """
        Returns the min energy.
        """
        return self._params[0]

    @property
    def b0(self):
        """
        Returns the bulk modulus.
        Note: the units for the bulk modulus: unit of energy/unit of volume^3.
        """
        return self._params[1]

    @property
    def b0_GPa(self):
        """
        Returns the bulk modulus in GPa.
        Note: This assumes that the energy and volumes are in eV and Ang^3
            respectively
        """
        return FloatWithUnit(self.b0, "eV ang^-3").to("GPa")

    @property
    def b1(self):
        """
        Returns the derivative of bulk modulus wrt pressure(dimensionless)
        """
        return self._params[2]

    @property
    def v0(self):
        """
        Returns the minimum or the reference volume in Ang^3.
        """
        return self._params[3]

    @property
    def results(self):
        """
        Returns a summary dict.

        Returns:
            dict
        """
        return dict(e0=self.e0, b0=self.b0, b1=self.b1, v0=self.v0)

    def plot(self, width=8, height=None, plt=None, dpi=None, **kwargs):
        """
        Plot the equation of state.

        Args:
            width (float): Width of plot in inches. Defaults to 8in.
            height (float): Height of plot in inches. Defaults to width *
                golden ratio.
            plt (matplotlib.pyplot): If plt is supplied, changes will be made
                to an existing plot. Otherwise, a new plot will be created.
            dpi:
            kwargs (dict): additional args fed to pyplot.plot.
                supported keys: style, color, text, label

        Returns:
            Matplotlib plot object.
        """
        plt = pretty_plot(width=width, height=height, plt=plt, dpi=dpi)

        color = kwargs.get("color", "r")
        label = kwargs.get("label", "{} fit".format(self.__class__.__name__))
        lines = ["Equation of State: %s" % self.__class__.__name__,
                 "Minimum energy = %1.2f eV" % self.e0,
                 "Minimum or reference volume = %1.2f Ang^3" % self.v0,
                 "Bulk modulus = %1.2f eV/Ang^3 = %1.2f GPa" %
                 (self.b0, self.b0_GPa),
                 "Derivative of bulk modulus wrt pressure = %1.2f" % self.b1]
        text = "\n".join(lines)
        text = kwargs.get("text", text)

        # Plot input data.
        plt.plot(self.volumes, self.energies, linestyle="None", marker="o",
                 color=color)

        # Plot eos fit.
        vmin, vmax = min(self.volumes), max(self.volumes)
        vmin, vmax = (vmin - 0.01 * abs(vmin), vmax + 0.01 * abs(vmax))
        vfit = np.linspace(vmin, vmax, 100)

        plt.plot(vfit, self.func(vfit), linestyle="dashed", color=color,
                 label=label)

        plt.grid(True)
        plt.xlabel("Volume $\\AA^3$")
        plt.ylabel("Energy (eV)")
        plt.legend(loc="best", shadow=True)
        # Add text with fit parameters.
        plt.text(0.4, 0.5, text, transform=plt.gca().transAxes)

        return plt


    @add_fig_kwargs
    def plot_ax(self, ax=None, fontsize=12, **kwargs):
        """
        Plot the equation of state on axis `ax`

        Args:
            ax: matplotlib :class:`Axes` or None if a new figure should be created.
            fontsize: Legend fontsize.
            color (str): plot color.
            label (str): Plot label
            text (str): Legend text (options)

        Returns:
            Matplotlib figure object.
        """
        ax, fig, plt = get_ax_fig_plt(ax=ax)

        color = kwargs.get("color", "r")
        label = kwargs.get("label", "{} fit".format(self.__class__.__name__))
        lines = ["Equation of State: %s" % self.__class__.__name__,
                 "Minimum energy = %1.2f eV" % self.e0,
                 "Minimum or reference volume = %1.2f Ang^3" % self.v0,
                 "Bulk modulus = %1.2f eV/Ang^3 = %1.2f GPa" %
                 (self.b0, self.b0_GPa),
                 "Derivative of bulk modulus wrt pressure = %1.2f" % self.b1]
        text = "\n".join(lines)
        text = kwargs.get("text", text)

        # Plot input data.
        ax.plot(self.volumes, self.energies, linestyle="None", marker="o", color=color)

        # Plot eos fit.
        vmin, vmax = min(self.volumes), max(self.volumes)
        vmin, vmax = (vmin - 0.01 * abs(vmin), vmax + 0.01 * abs(vmax))
        vfit = np.linspace(vmin, vmax, 100)

        ax.plot(vfit, self.func(vfit), linestyle="dashed", color=color, label=label)

        ax.grid(True)
        ax.set_xlabel("Volume $\\AA^3$")
        ax.set_ylabel("Energy (eV)")
        ax.legend(loc="best", shadow=True)
        # Add text with fit parameters.
        ax.text(0.5, 0.5, text, fontsize=fontsize, horizontalalignment='center',
            verticalalignment='center', transform=ax.transAxes)

        return fig

As we can see, there are two functions that we'll need to implement ourselves for our Murnaghan equation of state class: 
  
  - _func(self, volume, params):The equation of state function. 
  - fit(self)
    
for `_func`, you should translate the equation for the Murnaghan equation of state into a python function. for `params`, we input a list corresponding to [e0, b0, b1, v0]:

<img src="static/Murnaghan.png" alt="Murnaghan equation of state" width="300"/>

We will also be implementing a couple of other functions that will help you determine the phase transformation pressure.



In [37]:
# Our class, Murnaghan_EOS, inherits from EOSBase which has useful functions we can borrof for our EOS
class Murnaghan(EOSBase):
    """ A class representing a Murnaghan equation of state."""
    
    def _func(self, volume, params):
        """
        The equation of state function. This must be implemented by all classes
        that derive from this abstract class.

        Args:
            volume (float/numpy.array)
            params (list/tuple): values for the parameters other than the
                volume used by the eos [e0, b0, b1, v0]
        """
        e0 = params[0] # Note, this is not necessarily equal to self.e0
        b0 = params[1] # Note, this is not necessarily equal to self.b0
        b1 = params[2] # Note, this is not necessarily equal to self.b1
        v0 = params[3] # Note, this is not necessarily equal to self.v0
        
        #YOUR CODE HERE
        
    
    def fit(self):
        """
        Does least square fitting and returns the fit parameters.
        """
        
        # Define the objective function for a least squares fit (OBJ = E_DFT - EOS(V_))
        def objective_function(parameters, V, E_DFT):
            """ Objective function to optimize for least-squares fitting. 
            
            OBJ = EOS(V) - E_DFT
            
            (hint: EOS(V) is actually self._func(V, self.params))
            """
            # YOUR CODE HERE
            
            
        # Get initial guess for the params from the EOSBase class's _initial_guess method we inherited.
        self._params = self._initial_guess()
        
        # Use Scipy's least squares fitting function to fit our parameters to our EOS and 
        # get a parameter ierr from scipy to check that the fitting when well. 
        self.eos_params, ierr = scipy.optimize.leastsq(
            objective_func, self._params, args=(self.volumes, self.energies))
        
        # Set our new parameters from the lstsq fit, e0, b0, b1, v0
        self._params = self.eos_params
        
        if ierr not in [1, 2, 3, 4]:
            raise EOSError("Optimal parameters not found")

    
    # A function that computes E(V) using the fit EOS parameters
    def E(self, V):
        return self._func(V, self.eos_params)
    
    # Write a function that computes P(V) in terms of b0, b1, and v0 in GPa
    def P(self, V):
        b0 = self.b0
        b1 = self.b1
        v0 = self.v0
        
        return # YOUR CODE HERE
    
    # Write a function that computes the enthalpy in terms of self.P(V) and self.E(V) and V in eV/atom
    def H(self, V):
        return # YOUR CODE HERE
    
    # Plots an energy vs volume curve 
    def plot_energy(self, scaled=False, color="r"):
    
        plt.plot(self.volumes, self.energies, linestyle="None", marker="o",
                 color=color, label=self.name)
        # Plot eos fit.
        range = max(self.volumes) - min(self.volumes)
        vmin, vmax = min(self.volumes)-range*.5, max(self.volumes)+range*.5
    #     vmin, vmax = (vmin - 0.01 * abs(vmin), vmax + 0.01 * abs(vmax))
        vfit = np.linspace(vmin, vmax, 100)

        plt.plot(vfit, self.func(vfit), linestyle="dashed", color=color)
        plt.xlabel("Volume ($Å^3$/atom)")
        plt.ylabel("Energy (eV/atom)")
    
    # Plots an enthalpy vs volume curve 
    def plot_enthalpy(self, color="r"):
        plt.plot([self.P(v)*160.2 for v in self.volumes], [self.H(v) for v in self.volumes], linestyle="None", marker="o",
                 color=color, label=self.name)
        # Plot eos fit.
        range = max(self.volumes) - min(self.volumes)
        vmin, vmax = min(self.volumes), max(self.volumes)
        vfit = np.linspace(vmin, vmax, 1000)
        pfit = [self.P(v) for v in vfit]
        plt.plot(pfit, self.H(vfit), linestyle="dashed", color=color)
        plt.xlabel("Pressure (UNITS?)") # You should change "UNITS?" to appropriate units for your final plots
        plt.ylabel("Enthalpy (UNITS?)") # You should change "UNITS?" to appropriate units for your final plots
        
    
# Helper function to find the intersection of two lines
def line_intersection(f1, f2):
    return np.argwhere(np.isclose(f1, f2, atol=10)).reshape(-1)


Now, we can use our new `Murnaghan` class to fit our equation of state from the energies and volumes we calculated in the first part of this problem. (Make sure to divide by number of sites per unit cell to get E/atom and V/atom)

In [ ]:
DC_volumes = # YOUR CODE HERE
DC_energies = # YOUR CODE HERE
BSn_volumes = # YOUR CODE HERE
BSn_energies = # YOUR CODE HERE

DC = Murnaghan(DC_volumes, DC_energies, name="DC")
DC.fit()
DC.plot_energy()
BSn = Murnaghan(BSn_volumes, BSn_energies, name="Beta Tin")
BSn.fit()
BSn.plot_energy(color='b')
plt.legend()
plt.show()

We can also plot two EOS curves on the same axis, which might be helpful for finding the Equilibrium Pressure. 

In [ ]:
DC.plot_enthalpy()
BSn.plot_enthalpy(color='b')
plt.legend()